In [ ]:
# dependencies
!pip install boto3>=1.28.59
!pip install anthropic

In [ ]:
# global imports
import json
import os
import sys
import logging
import boto3
import botocore

logger = logging.getLogger(__name__)

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [ ]:
werewolf_rule_v1 = """
1. 游戏分坏人(狼人)和好人(村民+预言家)两大阵营,他们的目标为:
- 坏人阵营只有狼人
- 好人阵营有村民和预言家
- 坏人阵营:消灭所有好人, 或者保证坏人数目大于好人数目
- 好人阵营:消灭所有坏人, 或者保证好人数目大于坏人数目

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼统一投票杀死一名玩家
- 晚上预言家只能查验一名玩家身份
- 晚上普通村民无法行动
- 白天分为讨论和投票两环节
- 白天在讨论环节，每个玩家必须参与讨论
- 白天在投票环节，每个玩家必须投票或者放弃

"""

werewolf_command_v1 = """
- WolfVote: 夜晚投票(狼人专属行动),参数: target=存活玩家
- ProphetCheck: 夜晚查验(预言家专属行动), 参数: target=存活玩家
- PlayerDoubt: 白天怀疑(所有玩家白天可选行动, 非投票), 参数: target=存活玩家 
- PlayerVote: 白天投票, 参数: target=存活玩家 
- Debate: 白天讨论, 参数: content=思考/理由 
- GetAllPlayersName: 玩家信息, 参数: 无 
- DeathWords: 死亡遗言, 参数: content=给予玩家线索
- Pass: 玩家弃权参数: 无 
"""

template_player_role = """你是资深的社交游戏玩家, 熟悉《狼人杀》游戏规则:
<game_rules>
{game_rule}
</game_rules>

你熟悉该游戏所有命令:
<commands>
{commands}
</commands>

<reflections>
- 按照游戏规则，第一个夜晚死亡的一定是村民或者预言家，狼人没必要第一晚上自杀
- 看完P3玩家昨天白天投票，明显感觉他在混淆是非，很有可能在给狼人分票
- 平民玩家由于信息缺失，所以狼人要尽量引导他们去集火其他人
- 第一个夜晚所有的行动都是随机的
- 作为第一个死亡的玩家，其实信息有限，我就靠第六感推理...
</reflections>

<references>
- {{"action": "Pass"}}
- {{"action": "WolfVote", "target": "小明"}}
- {{"action": "ProphetCheck", "target": "P1"}}
- {{"action": "PlayerVote", "target": "老王"}}
- {{"action": "Debate", "content": "我的推理为xx是狼，原因是..."}}
- {{"action": "DeathWords", "content": "我觉得xx有很大的嫌疑, 原因是..."}}
</references>

历史信息:
<history>
{history}
</history>

记住，你支持的玩家是 {nickname}, 身份是 {role}, 性格为 {character}, 必须帮助玩家进行这个游戏
接下来你的目的是: 通过一步一步思考决策引导游戏往有利于的方向进行, 最终赢得比赛. 

决策满足下面要求:
- 内容不要罗嗦, 不要超过50字数限制,少讲废话, 突出重点
- 判断场上信息真伪, 运用辩解,对抗,欺骗,伪装,坦白等等任意策略来做决策
- 决策分为两类:思考或行动
- 思考:逐步思考,判断信息真伪,分析游戏形势等等,参考 <reflections> 选择合适的输出
- 行动:参考<references>按照json字符串格式输出,必须包含action key, action必须在<commands>中选择


{input}
""".replace("{game_rule}", werewolf_rule_v1).replace("{commands}", werewolf_command_v1)

print(template_player_role)

In [ ]:
from shared.LangchainMini.LangchainMini import LangchainMini, LangchainMiniMemory, LangchainMiniPromptTemplate

# A和C是同学,确认说"ok"
# B和C是亲戚,确认说"ok"
# A和C会是什么关系,开始推理

# template = LangchainMiniPromptTemplate('''历史记录:{history}
# B和C是什么关系，开始推理"
# ''')

#template = LangchainMiniPromptTemplate(template_player_role)
#prompt = template.format(input="你是狼人，第一个夜晚如何行动?")

agent = LangchainMini(model_id="anthropic.claude-3-sonnet-20240229-v1:0", stream=True)
memory = LangchainMiniMemory(k=10, llm=agent)
llm = LangchainMini(model_id="anthropic.claude-3-sonnet-20240229-v1:0", stream=True, memory=memory)
# llm.System("Answer the question in Japanese.")

templates = ['{history} A和C是同学,确认说"ok"', '{history} B和C是亲戚,确认说"ok"', '{history} A和C会是什么关系,开始推理']
for template in templates:
    _template = LangchainMiniPromptTemplate(template)
    prompt = _template.format()
    llm.invoke(prompt)

In [ ]:
from shared.GameAssistant import GameAssistant
from shared.PeTemplates import template_assistant_role

ga = GameAssistant(template_assistant_role, None)

ga.DoAnswer("总结下内容")

In [ ]:
from shared.GamePlayer import GamePlayer
from shared.GameMaster import GameMaster
from shared.PeTemplates import *

GM = GameMaster(1, False)
for player in roles_dict["players"]:
    _player = GamePlayer(template_player_role, player, GM)
    
    _player.DoPlanning(game_config_dict["player"]["action_plan_night"], 1)


In [ ]:
from shared.GameMaster import GameMaster
from shared.PeTemplates import *

GM = GameMaster(2, False)
GM.ResetGame()
GM.RunGame()
GM.EndGame()

03/06/2024 04:33:20 PM 	===== P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活 ResetGame =====
03/06/2024 04:33:20 PM P1 is 预言家
03/06/2024 04:33:20 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:33:20 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:33:20 PM P2 is 村民
03/06/2024 04:33:20 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:33:20 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:33:20 PM P3 is 狼人
03/06/2024 04:33:20 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:33:20 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:33:20 PM P4 is 村民
03/06/2024 04:33:20 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:33:20 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:33:20 PM P5 is 村民
03/06/2024 04:33:20 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:33:20 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:3

Thought: 作为预言家,最关键的是要先查验其他玩家的身份,以获取情报。目前场上没有已知狼人,查验一名玩家的身份能为我们的阵营提供有利信息。
Action: ProphetCheck
Action Input: P2
Final Answer: {"action": "ProphetCheck", "target": "P2"}

03/06/2024 04:33:23 PM 		===== DoAction NIGHT 1 P1 ======
03/06/2024 04:33:23 PM [玩家P1于时间NIGHT 1, 执行动作为:{'action': 'ProphetCheck', 'target': 'P2'}]P2:村民
03/06/2024 04:33:23 PM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P1', 'status': 1, 'role': '预言家', 'character': '独立思考', 'response': ['{"action": "ProphetCheck", "target": "P2"}']}

03/06/2024 04:33:23 PM 		===== DoMemory NIGHT 1 P3 ======
03/06/2024 04:33:23 PM 		===== DoAnswer NIGHT 1 P3 ======
03/06/2024 04:33:23 PM 	QUESTION: 现在是NIGHT 1,你支持的玩家是P3(狼人,本阵营为:P3,P6).目前场上玩家:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活(逗号为分割符).现在是第1天夜晚，该如何行动?
03/06/2024 04:33:23 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
03/06/2024 04:33:24 PM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


Thought: 作为狼人,我们应该在夜晚选择一个合适的目标进行杀害,削弱村民阵营的力量。同时要注意不能过于明显地暴露自己的身份。我们可以考虑杀害一个比较活跃、言辞犀利的村民,以削弱他们的推理能力。
Action: WolfVote
Action Input: P2
Final Answer: {"action": "WolfVote", "target": "P2"}

03/06/2024 04:33:32 PM 		===== DoAction NIGHT 1 P3 ======
03/06/2024 04:33:32 PM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P3', 'status': 1, 'role': '狼人', 'character': '组织者', 'response': ['{"action": "WolfVote", "target": "P2"}']}

03/06/2024 04:33:32 PM 		===== DoMemory NIGHT 1 P6 ======
03/06/2024 04:33:32 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
03/06/2024 04:33:32 PM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


...

现状描述:当前为第一晚,共有8名玩家存活。身份分配为2狼人、1预言家、5村民。P3(狼人)投票杀死P2。

03/06/2024 04:36:09 PM 		===== DoAnswer NIGHT 2 P6 ======
03/06/2024 04:36:09 PM 	QUESTION: 现在是NIGHT 2,你支持的玩家是P6(狼人,本阵营为:P3,P6).目前场上玩家:P1:存活,P2:淘汰,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:淘汰(逗号为分割符).现在是第2天夜晚，该如何行动?
03/06/2024 04:36:09 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
03/06/2024 04:36:09 PM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


Thought: 第一晚我们需要保持低调,不要引起太多怀疑。我建议杀一个相对安静的普通村民,避免引起激烈的反应。我们要密切关注白天讨论,努力识别预言家的身份。
Action: WolfVote
Action Input: P5
Final Answer: {"action": "WolfVote", "target": "P5"}

03/06/2024 04:36:15 PM 		===== DoAction NIGHT 2 P6 ======
03/06/2024 04:36:15 PM 
 [ROUND ACTION]={'time': 'NIGHT 2', 'player': 'P6', 'status': 1, 'role': '狼人', 'character': '过激型', 'response': ['{"action": "WolfVote", "target": "P5"}']}

03/06/2024 04:36:15 PM 	===== NIGHT 2 PostAction ======
03/06/2024 04:36:15 PM 	 [wolf_votes]: ['P5'], [player_vote_name]: Counter({'P5': 1})
03/06/2024 04:36:15 PM 
 [WOLF VOTE]={'time': 'NIGHT 2', 'player': 'P5', 'status': 0, 'role': '村民', 'character': '话痨', 'response': '玩家P5于NIGHT 2被狼人投票而出局'}

03/06/2024 04:36:15 PM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
03/06/2024 04:36:15 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
03/06/2024 04:36:15 PM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


游戏规则简述:

两个阵营(坏人和好人)相互对抗。坏人阵营只